## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2022-06-08 21:22:48,-0.7393,-90.3518,76.96,88,53,5.99
1,1,Langxiang,CN,2022-06-08 21:22:48,46.9500,128.8833,56.62,92,66,2.44
2,2,Ajaccio,FR,2022-06-08 21:22:49,42.0000,8.9167,69.85,68,40,4.61
3,3,Jacareacanga,BR,2022-06-08 21:22:49,-6.2222,-57.7528,81.34,71,100,2.21
4,4,Albany,US,2022-06-08 21:20:30,42.6001,-73.9662,75.79,49,23,1.01


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2022-06-08 21:22:48,-0.7393,-90.3518,76.96,88,53,5.99
2,2,Ajaccio,FR,2022-06-08 21:22:49,42.0000,8.9167,69.85,68,40,4.61
3,3,Jacareacanga,BR,2022-06-08 21:22:49,-6.2222,-57.7528,81.34,71,100,2.21
4,4,Albany,US,2022-06-08 21:20:30,42.6001,-73.9662,75.79,49,23,1.01
5,5,Bethel,US,2022-06-08 21:22:50,41.3712,-73.4140,83.01,46,0,5.75
8,8,Kapaa,US,2022-06-08 21:22:50,22.0752,-159.3190,83.48,73,0,8.05
9,9,Ixtapa,MX,2022-06-08 21:22:51,20.7000,-105.2000,79.83,78,100,10.36
13,13,Carutapera,BR,2022-06-08 21:22:52,-1.1950,-46.0200,78.03,86,76,7.11
15,15,Kieta,PG,2022-06-08 21:22:52,-6.2167,155.6333,78.48,83,98,3.80
16,16,Orbetello,IT,2022-06-08 21:22:52,42.4408,11.2196,71.62,66,100,4.03


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# OR
preferred_cities_df.notnull().sum()
# OR
preferred_cities_df.count()

City_ID       370
City          370
Country       369
Date          370
Lat           370
Lng           370
Max Temp      370
Humidity      370
Cloudiness    370
Wind Speed    370
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

KeyError: "['Current Description'] not in index"

In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")    

NameError: name 'hotel_df' is not defined

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
